<a href="https://colab.research.google.com/github/JuanFaria02/7-Day-Of-Code/blob/master/Sistema_de_Recomenda%C3%A7%C3%A3o(dia_4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introdução**


---

Grandes plataformas utilizam Machine Learning para criar sistemas de recomendação de acordo com cada usuário. As recomendações geram identificação do usuário com a plataforma portanto é muito importante que todo aplicativo use sistemas de recomendações robustos. Entretanto, esse projeto é apenas uma forma simplificada de introduzir esse tema para meus estudos. O projeto fará recomendações de filmes usando a base de dados do site [*MovieLens*](https://grouplens.org/datasets/movielens/) e recomendará com base na média das notas.

In [83]:
import pandas as pd
import numpy as np
import pickle

In [84]:
movies = pd.read_csv('/content/drive/MyDrive/Projetos /7 Days Of Code/Arquivo dados/ml-latest-small/movies.csv')
movies = movies.set_index('movieId')
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [85]:
ratings = pd.read_csv('/content/drive/MyDrive/Projetos /7 Days Of Code/Arquivo dados/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [87]:
ratings.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


Foi contada as frequências de votos para achar os filmes com mais votos e criado a tabela *total de votos* 

In [88]:
total_votos = ratings['movieId'].value_counts() #conta a frequência
#movies.loc[356] localiza pelo movie Id
total_votos.head()

356     329
318     317
296     307
593     279
2571    278
Name: movieId, dtype: int64

In [90]:
movies['total_de_votos'] = total_votos
movies.head()

,title,genres,total_de_votos
movieId,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0
2,Jumanji (1995),Adventure|Children|Fantasy,110.0
3,Grumpier Old Men (1995),Comedy|Romance,52.0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0
5,Father of the Bride Part II (1995),Comedy,49.0


A organização dos dados foi feita pelo total de votos 

In [91]:
movies = movies.sort_values('total_de_votos', ascending = False)


In [92]:
nota_media = ratings.groupby('movieId').mean()['rating']
nota_media.head()

movies['nota_media'] = nota_media

## Mais votados


---
Um dos meios que podem ser feitos recomendações é por poplaridade. Devido a isso, a primeira estratégia foi ver quais filmes tiveram mais votos para realizar as recomendações, porém, foi percebido que alguns filmes com nota média baixa entrou nela como por exemplo Jurassic Park com apenas 3.75 de nota média, visto isso, a utilização apenas de popularidade pode não ser o melhor caminho e usar as notas seria o mais apropriado. 


In [93]:
movies.head(10)

,title,genres,total_de_votos,nota_media
movieId,,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0,4.164134
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0,4.197068
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0,4.161290
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0,4.192446
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0,4.231076
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,238.0,3.750000
110,Braveheart (1995),Action|Drama|War,237.0,4.031646
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,224.0,3.970982


# Recomendar por média

Como exemplificado a cima, as recomendações apenas por votos não trouxe o resultado esperado, portanto, foi tomada a decisão de ordenar pelas médias. Porém, como mostra o próximo DataFrame as recomendações vieram com filmes pouco votados e de ninchos específicos sendo inapropriado para um público geral em que não se conhece os gostos e preferências. A solução para esse desafio foi mesclar a quantidade de votos com a nota média, trazendo as melhores recomendações para um público geral usando esse dataset.

### Tabela com poucos votos:

In [80]:
movies = movies.sort_values('nota_media', ascending = False)
movies.head()

,title,genres,total_de_votos,nota_media
movieId,,,,
6442,Belle époque (1992),Comedy|Romance,2.0,5.0
7071,"Woman Under the Influence, A (1974)",Drama,1.0,5.0
130978,Love and Pigeons (1985),Comedy|Romance,1.0,5.0
6021,"American Friend, The (Amerikanische Freund, De...",Crime|Drama|Mystery|Thriller,1.0,5.0
4116,Hollywood Shuffle (1987),Comedy,1.0,5.0


## Filmes com mais de 200 votos e com as maiores notas: 


In [97]:
filmes_recomendados = movies.query('total_de_votos >= 200').head(10)
filmes_recomendados = filmes_recomendados.sort_values('nota_media', ascending=False)
filmes_recomendados

,title,genres,total_de_votos,nota_media
movieId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0,4.231076
527,Schindler's List (1993),Drama|War,220.0,4.225000
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0,4.197068
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0,4.192446
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0,4.164134
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0,4.161290
110,Braveheart (1995),Action|Drama|War,237.0,4.031646
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,224.0,3.970982


In [99]:
with open('filmes_recomendados.pkl', 'wb') as file:
    pickle.dump(filmes_recomendados, file)
